In [1]:
import pandas as pd
from pandas.io import json
import datetime
import time

In [2]:
path = "../data/raw/purplesharp_ad_playbook_I_2020-10-22042947.json"
df = json.read_json(path, lines=True)

In [4]:
len(df)

25993

#### Kerberos Attack Related Features Explained (See paper on row 8 on google Sheets)  

AuthenticationPackageName: Indicates the authentication type. This could, for example, be Kerberos or NTLM.  
ElevatedToken: Whether or not the logon has elevated administrator privileges.  
ImpersonationLevel: The level of impersonation the logon is allowed to. Most common is to have impersonation privileges allowing to impersonate the client on the local system, but not remote systems.  
IpAddress: The source IP address where the logon originated from.  
IpPort: The source IP port of the remote system.  
KeyLength: The key length in bits for an NTLM authenticated logon.  
LmPackageName: The package name of an NTLM authenticated logon.  
LogonProcessName: The trusted logon process registered with the local security authority handling the logon.  
LogonType: The type of logon performed.  
ProcessName: The the executable of the process which initiated the logon.  
RestrictedAdminMode: Whether or not the logon was running in restricted admin mode.  
SubjectDomainName: The domain of the account initiating the logon.  
SubjectUserName: The username of the account initiating the logon.   
SubjectUserSid: The security identifer of the account initiating the logon.  
TargetDomainName: The domain the account being logged onto resides in.  
TargetOutboundDomainName: The domain for the user account being used for remote connections, in case of logon type 9.  
TargetOutboundUserName: The username of the account being used for remote connections.  
TargetUserName: The username of the account being logged onto.  
TargetUserSid: The security identier of the account being logged onto.  
WorkstationName: The computer name of the workstation from where the logon originated.  

In [3]:
print(df.columns.tolist())

['@timestamp', 'DestPort', 'ProcessId', 'OpcodeValue', 'port', 'SourceAddress', 'RemoteMachineID', 'Severity', 'SourcePort', 'EventTime', 'SourceName', 'ProviderGuid', 'Channel', 'DestAddress', 'FilterRTID', 'RemoteUserID', 'EventReceivedTime', 'ExecutionProcessID', 'EventID', 'Category', 'tags', 'SeverityValue', 'Keywords', 'EventType', 'Version', 'Direction', 'SourceModuleName', 'Application', 'SourceModuleType', 'host', 'LayerName', 'RecordNumber', 'ThreadID', 'Task', 'Message', 'Hostname', 'Protocol', 'LayerRTID', '@version', 'Opcode', 'AccountName', 'AccountType', 'Image', 'TargetFilename', 'UtcTime', 'Domain', 'RuleName', 'CreationUtcTime', 'UserID', 'ProcessGuid', 'TargetObject', 'EventTypeOrignal', 'Details', 'OriginalFileName', 'Product', 'Hashes', 'Signed', 'FileVersion', 'Company', 'SignatureStatus', 'Signature', 'ImageLoaded', 'Description', 'MandatoryLabel', 'SubjectUserSid', 'SubjectUserName', 'CommandLine', 'SubjectDomainName', 'SubjectLogonId', 'ParentProcessName', 'Tar

In [14]:
df.isnull().sum()

@timestamp             0
DestPort           25796
ProcessId           1258
OpcodeValue          342
port                   0
                   ...  
MiniportNameLen    25991
MiniportName       25991
param3             25992
param4             25992
Time                   0
Length: 195, dtype: int64

#### Finding Columns similar to Lanl dataset  
// To Confirm

Lanl is another AD dataset for benign behavior that we might consider merging with - See 2.0-vb-initial-lanl-benign-data-exploration.ipynb  

'AuthenticationPackage',    
'Destination',  
'DomainName',    
'EventID',    
'FailureReason',  
'LogHost',   
'LogonID',  
'LogonType',  
'LogonTypeDescription',  
'ParentProcessID',   
'ParentProcessName',  
'ProcessID',  
'ProcessName',  
'ServiceName',  
'Source',  
'Status',  
'SubjectDomainName',  
'SubjectLogonID',  
'SubjectUserName',  
'Time',  
'UserName'  

In [5]:
df_lanl = df[['EventID','EventTime','LogonId', 'host', 'Hostname', 'LogonType' ,'SubjectUserName', 'SubjectDomainName',
    'SubjectLogonId','Status', 'SourceName', 'ServiceName',"ServiceSid",'DestinationHostname', 'AuthenticationPackageName',
    'FailureReason','ProcessName', 'ProcessId', 'ParentProcessName','ParentProcessId'
   ]]

In [5]:
df_lanl.isnull().all(0)

EventID                      False
EventTime                    False
LogonId                      False
host                         False
Hostname                     False
LogonType                    False
SubjectUserName              False
SubjectDomainName            False
SubjectLogonId               False
Status                       False
SourceName                   False
ServiceName                  False
ServiceSid                   False
DestinationHostname          False
AuthenticationPackageName    False
FailureReason                False
ProcessName                  False
ProcessId                    False
ParentProcessName            False
ParentProcessId              False
dtype: bool

In [6]:
df_lanl

,EventID,EventTime,LogonId,host,Hostname,LogonType,SubjectUserName,SubjectDomainName,SubjectLogonId,Status,SourceName,ServiceName,ServiceSid,DestinationHostname,AuthenticationPackageName,FailureReason,ProcessName,ProcessId,ParentProcessName,ParentProcessId
0,5156,2020-10-22 04:29:46,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Security-Auditing,NaN,NaN,NaN,NaN,NaN,NaN,696,NaN,NaN
1,5156,2020-10-22 04:29:46,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Security-Auditing,NaN,NaN,NaN,NaN,NaN,NaN,3632,NaN,NaN
2,5156,2020-10-22 04:29:46,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Security-Auditing,NaN,NaN,NaN,NaN,NaN,NaN,696,NaN,NaN
3,5156,2020-10-22 04:29:46,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Security-Auditing,NaN,NaN,NaN,NaN,NaN,NaN,696,NaN,NaN
4,5156,2020-10-22 04:29:46,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Security-Auditing,NaN,NaN,NaN,NaN,NaN,NaN,3632,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25988,12,2020-10-22 04:30:01,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN,7944,NaN,NaN
25989,12,2020-10-22 04:30:01,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN,7944,NaN,NaN
25990,12,2020-10-22 04:30:01,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN,7944,NaN,NaN
25991,12,2020-10-22 04:30:01,NaN,wec.internal.cloudapp.net,MORDORDC.theshire.local,NaN,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN,7944,NaN,NaN


In [8]:
for col in list(df_lanl):
    print(col)
    print(df_lanl[col].unique())

EventID
[ 5156  5158    11    12    13     7  4688  4690     1  4658    10  4656
  4663  4673  4703  4672  4624  4627  4634   600   400    23    17   800
  5857 53504  4105  4106  4103  4104  4625  4670  4776  4769  5140  5145
  8193  8194  8195  8197    22     3    18  4674     9  4799  8196 12039
 40961 40962   403  4689  4661     5  5058  5061  7040]
ProcessName
[nan 'C:\\Users\\pgustavo\\Downloads\\PurpleSharp.exe'
 'C:\\Windows\\System32\\svchost.exe'
 'C:\\Windows\\System32\\wbem\\WmiPrvSE.exe' '-'
 'C:\\Windows\\System32\\SearchProtocolHost.exe'
 'C:\\Windows\\System32\\SearchIndexer.exe'
 'C:\\Windows\\System32\\SearchFilterHost.exe'
 'C:\\Windows\\System32\\lsass.exe' 'C:\\Windows\\System32\\dllhost.exe'
 'C:\\Windows\\System32\\wsmprovhost.exe'
 'C:\\Windows\\System32\\services.exe'
 'C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe'
 'C:\\Windows\\System32\\conhost.exe']
Time
[34186. 34187. 34189. 34192. 34193. 34194. 34195. 34196. 34197. 34198.
 34199. 34200. 

In [9]:
df_trimmed = df[['@timestamp',
      'SourceAddress',
      'RemoteMachineID',
      'EventTime',
      'EventID', 
#       'Category',
      'EventType',
      'UserID',
      'AccountName',
#       'Domain', 
#       'RuleName',
#       'Description',
#       'NewProcessName', 
#       'NewProcessId',
      'LmPackageName',
      'Service',
      'LogonProcessName', 
      'WorkstationName',
      'LogonType',
      'EventCountTotal',
      'Workstation',
#       'FailureReason', 
#       'Status',
#       'ServiceName', 
#       'ServiceSid', 
#       'TicketOptions'      
     ]]

In [10]:
for col in list(df_trimmed):
    print(col)
    print(df_trimmed[col].unique())

@timestamp
['2020-10-22T08:29:48.785Z' '2020-10-22T08:29:48.786Z'
 '2020-10-22T08:29:49.785Z' ... '2020-10-22T08:30:42.830Z'
 '2020-10-22T08:30:42.831Z' '2020-10-22T08:30:42.832Z']
SourceAddress
['::1' '0.0.0.0' '172.18.39.7' nan '127.0.0.1' '172.18.38.5' '172.18.39.5'
 '::' '172.18.39.6' '172.18.38.6' 'fe80::a1cd:665f:9ac2:55c1']
RemoteMachineID
['S-1-0-0' nan]
EventTime
['2020-10-22 04:29:46' '2020-10-22 04:29:47' '2020-10-22 04:29:49'
 '2020-10-22 04:29:52' '2020-10-22 04:29:53' '2020-10-22 04:29:54'
 '2020-10-22 04:29:55' '2020-10-22 04:29:56' '2020-10-22 04:29:57'
 '2020-10-22 04:29:58' '2020-10-22 04:29:59' '2020-10-22 04:30:00'
 '2020-10-22 04:30:01' '2020-10-22 04:30:02' '2020-10-22 04:30:03'
 '2020-10-22 04:30:04' '2020-10-22 04:30:05' '2020-10-22 04:30:06'
 '2020-10-22 04:30:07' '2020-10-22 04:30:08' '2020-10-22 04:30:09'
 '2020-10-22 04:30:10' '2020-10-22 04:30:11' '2020-10-22 04:30:14'
 '2020-10-22 04:30:17' '2020-10-22 04:30:19' '2020-10-22 04:30:21'
 '2020-10-22 04:30:23'

In [11]:
df['Time'] = pd.to_datetime(df['EventTime']) - pd.DateOffset(years = 50, months = 9 , days=22) 

In [12]:
df.count()

@timestamp         25993
DestPort             197
ProcessId          24735
OpcodeValue        25651
port               25993
                   ...  
MiniportNameLen        2
MiniportName           2
param3                 1
param4                 1
Time               25993
Length: 195, dtype: int64

In [13]:
len(df[(df.EventID == 4672)])

43

In [2]:
path = "../data/raw/purplesharp_ad_playbook_I_2020-10-22042947.json"
df = json.read_json(path, lines=True)

In [3]:
event_list = [1, 3, 8, 10, 11, 12, 4624, 4625, 4648, 4658, 4661, 4663, 4672,4698,4768,5140,5145, 5156, 5158]
df[(df.EventID.isin(event_list))]['EventID'].value_counts()
# df['EventID'].value_counts()

12      17917
4658      761
11        629
10        345
4663      250
5156      197
5158      129
3         126
4624       49
4672       43
1          24
4625        7
5145        3
5140        3
4661        2
Name: EventID, dtype: int64

In [4]:
path = "../data/raw/empire_uac_shellapi_fodhelper_2020-09-04032946.json"
df = json.read_json(path, lines=True)

In [5]:
df[(df.EventID.isin(event_list))]['EventID'].value_counts()
#df['EventID'].value_counts()

10      948
12      858
4658    202
4663     67
5156     55
5158     40
11       34
1        18
3        12
4672      4
4624      4
5145      1
5140      1
Name: EventID, dtype: int64

In [6]:
path = "../data/raw/purplesharp_pe_injection_createremotethread_2020-10-2303121085.json"
df = json.read_json(path, lines=True)

In [7]:
df[(df.EventID.isin(event_list))]['EventID'].value_counts()
#df['EventID'].value_counts()

10      253
4658     46
12       15
5156     12
4663      7
1         6
11        3
5158      2
8         1
Name: EventID, dtype: int64

In [8]:
path = "../data/raw/schtask_create_2020-12-1907003032.json"
df = json.read_json(path, lines=True)

In [9]:
df[(df.EventID.isin(event_list))]['EventID'].value_counts()
#df['EventID'].value_counts()

10      404
4658     28
12       28
5156     17
5158     15
4663     14
11       10
1        10
3         6
4698      1
4672      1
4624      1
8         1
Name: EventID, dtype: int64